In [1]:
import pickle
import glob
import numpy as np
from collections import Counter
import pandas as pd
import librosa

In [2]:
# this is how you save the model to pickle 
# from keras.models import load_model

# new_model = load_model('Fri_May_17_22-52-09_2019_2dcnn_spec_.h5')
# pkl_filename = "aihua_model.pkl"  
# with open(pkl_filename, 'wb') as file:  
#     pickle.dump(new_model, file)

In [2]:
pkl_filename = "aihua_model.pkl"  
with open(pkl_filename, 'rb') as file:  
    pickle_model = pickle.load(file)
print(type(pickle_model))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
<class 'keras.engine.sequential.Sequential'>


In [4]:
def get_subarrays(row):
    return [row[pos:pos + 3,] for pos in range(0, len(row),3)]

In [5]:
labels = {'Saxophone': 0,
 'Glockenspiel': 1,
 'Cello': 2,
 'Knock': 3,
 'Gunshot_or_gunfire': 4,
 'Hi-hat': 5,
 'Laughter': 6,
 'Flute': 7,
 'Telephone': 8,
 'Bark': 9,
 'Scissors': 10,
 'Gong': 11,
 'Microwave_oven': 12,
 'Shatter': 13,
 'Harmonica': 14,
 'Bass_drum': 15,
 'Oboe': 16,
 'Bus': 17,
 'Tambourine': 18,
 'Keys_jangling': 19,
 'Electric_piano': 20,
 'Clarinet': 21,
 'Fireworks': 22,
 'Meow': 23,
 'Double_bass': 24,
 'Cough': 25,
 'Acoustic_guitar': 26,
 'Violin_or_fiddle': 27,
 'Snare_drum': 28,
 'Squeak': 29,
 'Finger_snapping': 30,
 'Writing': 31,
 'Trumpet': 32,
 'Drawer_open_or_close': 33,
 'Cowbell': 34,
 'Tearing': 35,
 'Fart': 36,
 'Chime': 37,
 'Burping_or_eructation': 38,
 'Computer_keyboard': 39,
 'Applause': 40}

inverted_mapping = {v:k for k,v in labels.items()}

In [6]:
def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10),min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [7]:
path = 'audio_test/'
def get_predicted_label(row):
    try:
        sig, fs = librosa.load(path + row)
        mask = envelope(sig,fs,0.0005)
        new_sig = sig[mask]
        S = librosa.feature.melspectrogram(y=new_sig, sr=fs)
        np.save("new.npy", S)
        data = np.load("new.npy").T
        data_slice = get_subarrays(data)
        xs = []
        for array_set in data_slice:
            if array_set.shape == (3,128):  # The network take input of shape (3, 128, 1)
                xs.append(array_set)
        features = np.asarray(xs)
        xs_reshape = features.reshape(features.shape[0], 3, 128, 1)
        pred_labels = [inverted_mapping.get(x) for x in pickle_model.predict_classes(xs_reshape)] # going to return a prediction for each of the 33
        label = Counter(pred_labels).most_common(1)[0][0]
        return label
    except Exception as e:
        return str(e)

In [8]:
test_files = glob.glob(path+'*.wav')
test_file = test_files[76].split('/')[1]
test_file

'118f1763.wav'

In [9]:
get_predicted_label(test_file)

'Clarinet'

In [10]:
test_audios = glob.glob('audio_test/' +'*.wav')
len(test_audios)
file_names = [x.split("/")[1] for x in test_audios]
print(len(file_names))
df = pd.DataFrame(file_names)
df.rename(columns={0: "fileName"},inplace=True)
df.head()

9400


,fileName
0,8319139c.wav
1,003995fa.wav
2,9c80c2dc.wav
3,93dbba2e.wav
4,45fbe67a.wav


In [13]:
import multiprocessing as mp
from multiprocessing import Pool

In [14]:
num_partitions = 10
num_cores = 4 

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
def multiply_columns(data):
    data['label'] = data['fileName'].apply(get_predicted_label)
    return data
    
dfs = parallelize_dataframe(df, multiply_columns)

In [ ]:
# cores = cpu_count() 